# Cancer experiment

In [1]:
def wmean_qc_coef(wmean, threshold, extended_list, true_genes):
    wmean_names = wmean['gene_symbol'].values
    pred_y = np.array(wmean['Probability'].values >= threshold)
    new_y = np.array(list(map(lambda gene: (gene in extended_list['gene_symbol'].values) & \
                        (gene not in true_genes['gene_symbol'].values), wmean_names))) #wmean_names
    
    if np.sum(pred_y) == 0:
        return 0
    Pr = np.sum(pred_y) / pred_y.size
    TP = np.sum(new_y & pred_y) # TP
    FN = np.sum(new_y) - TP # FN
    recall = TP/(TP+FN)
    coef = recall**2/Pr
    norm_coef = coef * np.sum(new_y) / new_y.size
    return coef # coef
    
def perform_qc(wmean, extended_list, true_genes):
    threshold_range = wmean.Probability.values #[0.01, 0.1] + [i for i in range(1, 98)]
    qc_range = [wmean_qc_coef(wmean, threshold, extended_list, true_genes) for threshold in threshold_range]
    #print(qc_range)
    optimal_thr = [threshold_range[i] for i, j in enumerate(qc_range) if j == max(qc_range)][0]
    qc_coef = wmean_qc_coef(wmean, optimal_thr, extended_list, true_genes)
    print('qc_coef:', qc_coef)
    print('opt_threshols:', optimal_thr)
    return qc_range

In [2]:
from sklearn.cluster import FeatureAgglomeration
from sklearn.preprocessing import RobustScaler
import pandas as pd
from ensemble import *

In [10]:
ntrain = [round(212*dob/100) for dob in range(2,20,2)]
ntrain

[4, 8, 13, 17, 21, 25, 30, 34, 38]

In [6]:
y.sum()

2

In [12]:
import math
from sklearn.preprocessing import StandardScaler

roc_dict = dict()
f_dict = dict()
pu_dict = dict()


for ntr in  ntrain:
    
    roc_store = list()
    f1_store = list()
    pu_store = list()
    
    print(f'NTR ==== {ntr}')
    for i in range(10):
        X = pd.read_csv('/home/nikita/Downloads/cancer.csv')
        y = X.diagnosis
        del X['diagnosis']
        del X['Unnamed: 32']
        del X['id']
        
        scaler = StandardScaler()
        scaler.fit(X)
        X = pd.DataFrame(scaler.transform(X))
        y = y.map(lambda x: 1 if x == 'M' else 0)
        hidden_size = y.sum() - ntr
        y_orig = y.copy()
        y.loc[
                np.random.choice(
                    y[y == 1].index, 
                    replace = False, 
                    size = hidden_size
                )
            ] = 0
        val_y = y[y == 1].copy()
        y.loc[
                np.random.choice(
                    y[y == 1].index, 
                    replace = False, 
                    size = round(ntr*1/3)
                )
            ] = 0
        als = pd.DataFrame({'gene_symbol':list(set(y_orig.index-y.index))})
        X['gene_symbol'] = X.index
        ens = EnsembleClassifier(X, y, MODELS, als, n_bootstrap=60)
        ens.run_estimators()
        wmean = ens.best_scored_proba()
        wmean['Probability'] = wmean.Probability.apply(lambda x: 100 if x==-1 else x)
        extended_list = pd.DataFrame({'gene_symbol':y_orig[y_orig==1].index.values})
        true_genes = pd.DataFrame({'gene_symbol':val_y.index.values})
        qc = max(perform_qc(wmean, extended_list, true_genes))
        pu_store.append(qc)
    pu_dict[str(f'{ntr}')] = pu_store

NTR ==== 4


StandardScaler(copy=True, with_mean=True, with_std=True)

Logistic regression	
PU-score: 21.884615384615383

Support Vector Machine	
PU-score: 1.2262931034482758

ADABoosting	
PU-score: 94.83333333333333

Random Forest	
PU-score: 40.642857142857146

Decision Tree	
PU-score: 142.25

Finding best combination	
Best combination is: ['Logistic regression' 'ADABoosting' 'Decision Tree']
PU-score: 142.25
qc_coef: 2.359581969182711
opt_threshols: 29.1011257468666


StandardScaler(copy=True, with_mean=True, with_std=True)

Logistic regression	
PU-score: 7.586666666666667

Support Vector Machine	
PU-score: 1.0053003533568905

ADABoosting	
PU-score: 12.369565217391305

Random Forest	
PU-score: 10.537037037037038

Decision Tree	
PU-score: 10.735849056603774

Finding best combination	
Best combination is: ['ADABoosting' 'Random Forest']
PU-score: 12.931818181818182
qc_coef: 2.267500721604849
opt_threshols: 45.35169491525423


...

PU-score: nan

Random Forest	


/home/nikita/Documents/work/git_projects/GPrior/qctoolbox.py:14: RuntimeWarning: invalid value encountered in long_scalars
  recall = tp/(tp+fn)


PU-score: nan

Decision Tree	
PU-score: nan

Finding best combination	


/home/nikita/Documents/work/git_projects/GPrior/qctoolbox.py:14: RuntimeWarning: invalid value encountered in long_scalars
  recall = tp/(tp+fn)


UnboundLocalError: local variable 'index_max' referenced before assignment

In [10]:
pu_dict.keys()

dict_keys(['3', '5', '8', '11', '14', '17'])

In [27]:
3 in extended_list['gene_symbol'].values

True

In [25]:
wmean['gene_symbol']

3        3
9        9
78      78
5        5
352    352
      ... 
351    351
215    215
487    487
214    214
7        7
Name: gene_symbol, Length: 569, dtype: int64

In [36]:
val_y

30     1
54     1
100    1
138    1
215    1
282    1
414    1
441    1
533    1
535    1
Name: diagnosis, dtype: int64